In [56]:
import pandas as pd
from IPython.display import clear_output
from bs4 import BeautifulSoup
import sys, os
import codecs
from bs4 import BeautifulSoup
import codecs 
import numpy as np 
from datetime import timedelta
import warnings
import numpy as np 
from IPython.display import clear_output
from collections import defaultdict
from dateutil import parser
import pandas as pd 
from time import sleep
import requests
from bs4 import BeautifulSoup
import urllib.parse as urlparse
from urllib.parse import parse_qs
from IPython.core.display import display, HTML

import os 
from collections import defaultdict

import numpy as np
import pysal as ps 
import utm 
import geopandas 
warnings.filterwarnings('ignore')

In [21]:

def grab_source(letter,path):

  
    """
    Queries the Real Property Parcer of erie county(Specifically only SBL values), which can be found on this link:
                            http://www2.erie.gov/ecrpts/index.php?q=real-property-parcel-search
    This function will return a beautiful soup object of the source code of a SBL's tax History.
    :param sbl: SBL value that we are searching for
    :param mode: it can be tax information history for: "ecgov", or owner history for anything else
    :return: a Beautiful soup object of the tax extry history of that SBL
    """
    path= os.path.join(path, 'SCRAPED_DATA_property_information')
    path2 = os.path.join(path, 'SCRAPED_DATA_owner_history')
    os.mkdir(path)
    os.mkdir(path2)
    body = {'txtsbl': letter, 
            'Juris': '1402',# WE CAN CHANGE THIS TO SBL BUT THIS IS OWNER NAME
            'showHistory': 'y'}
    url2 = 'https://paytax.erie.gov/webprop/'  # url we needed adding to since this is the root link
    url = 'https://paytax.erie.gov/webprop/property_info_results.asp'  # search website
    s_request = requests.Session()
    link = s_request.post(url, data=body)
    source_soup = BeautifulSoup(link.content, 'html.parser')
    while(len(source_soup.find_all('a', href=True)) > 1):
        for owner in source_soup.find_all('a', href=True):
            if str(owner['href']).startswith("property_info_details"): 
                con2 = requests.post(url2+owner['href'])
                details_soup = BeautifulSoup(con2.content, 'lxml')
                html_file = urlparse.urlparse(owner['href']).query.split("&")[0].replace('.','#').replace('/','+') + ".html"
                with open(path2 +  html_file, "w", encoding='utf-8') as file:
                            file.write(str(details_soup)) 
                for a in details_soup.find_all('a', href=True):
                    if str(a['href']).startswith('property_info_history.asp'):
                        s = BeautifulSoup(requests.post(url2 + a['href']).content,'lxml')
                        with open(path +  html_file, "w", encoding='utf-8') as file:
                            file.write(str(s)) 
                            return html_file
            elif str(owner['href']).startswith("property_info_results_next"):
                con3 = s_request.post(url2+owner['href'],headers= headers)
                source_soup = BeautifulSoup(con3.content, 'lxml')
        if len(source_soup.find_all(lambda tag: "Last Record Found" in tag.string if tag.string else False)) > 0: # that means this page is the last record 
            break



In [22]:
def scrape_considerations_page(owner_history_df,path):
    """
        Queries the Real Property Parcer of erie county(Specifically only SBL values), which can be found on this link:
                                http://www2.erie.gov/ecrpts/index.php?q=real-property-parcel-search
        This function will return a beautiful soup object of the source code of a SBL's tax History.
        :param sbl: SBL value that we are searching for
        :param mode: it can be tax information history for: "ecgov", or owner history for anything else
        :return: a Beautiful soup object of the tax extry history of that SBL
    """

    path= os.path.join(path, 'considerations_htmls')
    os.mkdir(path)
    counter = 0
    for sbl, deed_page in owner_history_df[['SBL','Book-Page']].to_records(index=False):
        try:
            file_name = sbl + "+" + deed_page 
            print("FILE:" + file_name)
            file_name = file_name.replace('.','#').replace('/','$')
            body = {'search_entry': deed_page,  # WE CAN CHANGE THIS TO SBL BUT THIS IS OWNER NAME
                    'search_by': 'Dynamic Book/Page'}
            headers = {'User-Agent': 'Mozilla/5.0'}

            url2 ='http://ecclerk.erie.gov/or_wb1/'  # url we needed adding to since this is the root link

            url = 'http://ecclerk.erie.gov/or_wb1/new_sch.asp'  # search website
            link = requests.post(url, data=body,headers = headers )
            source_soup = BeautifulSoup(link.content, 'html.parser')
            owner = source_soup.find_all('a', {'class': 'list_2'})
            counting = 1 
            for own in owner:
                con2 = requests.post(url2+ ''.join(own['href'].split()))
                details_soup = BeautifulSoup(con2.content, 'lxml')    
                with open(path+file_name+"&" + str(counting) + ".html","w") as of:
                    of.write(str(details_soup.html)) 
                counting += 1
            counter += 1 
        except Exception as e:
            print('failure on----------------------')
            print(sbl)
            print(deed_page)
            print(e)
            #lst_failed.append([deed_page,sbl])
        print(counter/len(df2) * 100)
        clear_output(wait = True)
            #print(pd.read_html(str(details_soup))[4][1][6])

In [23]:
def html_csv_converter(files,csv_name):
    
    counter = 0
    lst_failed =[] 
    for file_name in files:
        x= file_name.replace('#','.').replace('+','/')[:-5]
        try:
            with open(path +  file_name, "r", encoding='utf-8') as file:
                        data= file.read()

            df= pd.read_html(str(data))[0]
            df['SBL'] = file_name.replace('#','.').replace('+','/')[:-5]
            df.to_csv('../../../data/gentrification/LATEST/' + csv_name , mode='a', header=False,index = False)
            counter += 1 
        except:
            lst_failed.append(file_name)
            pass

In [32]:
def owner_history_error_reformat(owner_history_df,open_data_df, inplace = False):

    '''
    use open data informtation to replace errors 
    and noise within the original data 
    '''
    if not inplace:
        owner_history_df = owner_history_df.copy()
    owner_history_deeds = owner_history_df['Book-Page/Date identification'].str.split('*').str[0].str.replace(' ','').str.replace('-','/')
    owner_history_df['Book-Page'] = owner_history_deeds
    index_of_nan = owner_history_deeds[owner_history_deeds.apply(lambda x: True if x != x else False )].index
    index_of_errors = owner_history_deeds.loc[list(index_of_nan)].index

    nan_removed_owner_history = owner_history_deeds.loc[np.delete(owner_history_deeds.index,index_of_errors)]
    parsing_errors = nan_removed_owner_history[nan_removed_owner_history.apply(lambda x: '/' not in x)].index
    all_errors = np.append(index_of_nan,parsing_errors)
    owner_history_deeds.loc[parsing_errors] = owner_history_df.loc[parsing_errors]['Book-Page/Date identification'].str.split().apply(lambda i: i[0]).str.replace('-','/').str.replace('*','')

    open_data_df['Book-Page/Date identification'] = open_data_df[['DEED BOOK', 'DEED PAGE']].apply(lambda x: str(x[0]) + "/" + str(x[1]), axis =1 )
    new_deeds_from_ODB = open_data_df[['Book-Page/Date identification','PRINT KEY']].set_index('PRINT KEY').to_dict()['Book-Page/Date identification']
    owner_history_df.loc[all_errors,'Book-Page'] = owner_history_df.loc[all_errors]['SBL'].map(new_deeds_from_ODB)
    return owner_history_df

In [25]:
def combine_considerations_owner_history(file_path,csv_name):
   
    appended_data_frame = []
    counter = 0
    lst_failed= []
    dct = defaultdict(list)
    files = os.listdir(file_path)
    for file_name in files: 
        with open(file_path + file_name, "r", encoding='utf-8') as file:
                data = file.read()
        name = fl[:-5].replace('#','.').replace('$','/')
        sbl = name.split("+")[0]
        deed_page = name.split("+")[1].split("&")[0]
        doc_types = []
        try:
           
            consideration_page_df= pd.read_html(str(data))
        except Exception as e:
            lst_failed.append([fl,e])
            lst.append([sbl,np.nan,np.nan,deed_page,np.nan,np.nan])
            counter += 1 
            continue 

        doc_type = consideration_page_df[4][1][0]
        consideration = consideration_page_df[4][1][6]
        date = consideration_page_df[4][1][2]
        url_book_page = consideration_page_df[4][1][4]
        dct_name = sbl + "+" + deed_page
        print(counter/len(files)*100)
        if doc_type not in dct[dct_name]:
            appended_data_frame.append([sbl,consideration,doc_type,deed_page,url_book_page,date])
            dct[dct_name].append(doc_type)
        counter += 1 
        clear_output(wait = True) 
        df = pd.DataFrame(appended_data_frame, columns = ['SBLS','considerations','doc type','deed page(search parcel)','deed page(public records)','date(public records)'])
        df.to_csv(os.path.join(file_path,csv_name),index = False)
        return lst_failed

In [28]:
def weights_initialization(df_data,column_name_of_locations):
    '''
    Initializes neighborhood weights 
    '''

    df_data = data_frame_construction_for_rf(df_data)
 
    #-------------------------------------------------
    sbls = df_data.drop_duplicates('sbl')[['sbl',column_name_of_locations]]
    locations_sbls = sbls[column_name_of_locations].apply(lambda x: [ float(n.replace('(','').replace(')',''))  for n in x.split(',')])
    sbls['locs'] = locations_sbls.apply(lambda pt: utm.from_latlon(pt[0],pt[1])[:2])
    sbls['utm'] = sbls['locs'].apply(lambda pt: [pt[0],pt[1]])
    unique_sbls = df_data['sbl'].unique()
    res_loc = geopandas.GeoDataFrame(sbls, geometry=geopandas.points_from_xy(sbls['locs'].apply(lambda x: x[0]), sbls['locs'].apply(lambda y: y[1])))
    res_loc = res_loc.reset_index(drop = True)
    tester = geopandas.GeoDataFrame(res_loc).reset_index(drop = True)
    w300=ps.lib.weights.DistanceBand.from_dataframe(tester,threshold =300, ids = 'sbl', binary = False, p = 2)
    w500=ps.lib.weights.DistanceBand.from_dataframe(tester,threshold =500, ids = 'sbl', binary = False, p = 2)
    w80 =ps.lib.weights.DistanceBand.from_dataframe(tester,threshold =100, ids = 'sbl', binary = False, p = 2)
    return w300,w500,w80, df_data, unique_sbls 

In [68]:
def random_classifier_data_pre_processing(data_frame,open_dat):

    # permits data acess 
    
    permits_data =  pd.read_csv('https://data.buffalony.gov/api/views/9p2d-f3yt/rows.csv?accessType=DOWNLOAD', low_memory= False)
    
    permits_data['ISSUED'] = pd.to_datetime(permits_data['ISSUED'])
    permits = open_dat[['SBL','PRINT KEY',]].merge(permits_data, on = 'SBL', how = 'right')
    permits = permits.rename(columns = {'PRINT KEY':'sbl'}).drop('SBL',axis =1 )
    
    # -----------------------------------------------------------------
    # -------  Weights and data initialization ------------------------
    data_frame = data_frame.copy()
    to_dataframe = defaultdict(list)
    counter = 0
    list_of_neighborhoods = data_frame['Neighborhood'].unique()[:-1]
    w500, w300, w80, data_frame, unique_sbls = weights_initialization(data_frame, 'Location')
    dates_six = pd.date_range(start="1993-12-01",end="2021-12-01", freq='A-DEC')[1:]
    last_date = dates_six[0]
    
    # -------- loop feature collection --------------------------------
    
    for dates in dates_six[1:]: # we start from the second to the first date
        timedelta_days = (dates - parser.parse('1854-10-01')).days
        three_months = last_date + timedelta(90)
        last_year = last_date - timedelta(365)
        last_2year = last_date - timedelta(730)
        last_3year = last_date - timedelta(1095)
        last_45_year = last_date - timedelta(1825)
        #------
        length = data_frame[data_frame['Actual Date'] > last_date][data_frame['Actual Date'] <= dates]
        sbls_bought_this_year =  np.array(length['sbl']) # drop duplicates dates here just to be safe 
        #----
        length_3month = data_frame[data_frame['Actual Date'] > last_date][data_frame['Actual Date'] <= three_months]
        sbls_bought_3month =  np.array(length_3month['sbl'])
        #-----
        length_last_year = data_frame[data_frame['Actual Date'] >= last_year][data_frame['Actual Date'] <= last_date]
        sbls_bought_last_year =  set(np.array(length_last_year['sbl']))
        perm_last_year = permits[permits['ISSUED'] >= last_year][permits['ISSUED'] <= last_date]
        #----
        length_last_2year = data_frame[data_frame['Actual Date'] >= last_2year][data_frame['Actual Date'] <= last_year]
        sbls_bought_last_2year =  np.array(length_last_2year['sbl'])
        perm_last_2year = permits[permits['ISSUED'] >= last_2year][permits['ISSUED'] <= last_year]
        # -- 
        length_last_3year = data_frame[data_frame['Actual Date'] >= last_3year][data_frame['Actual Date'] <= last_2year]
        sbls_bought_last_3year =  np.array(length_last_3year['sbl'])
        perm_last_3year = permits[permits['ISSUED'] >= last_3year][permits['ISSUED'] <= last_2year]
        # -- 
        length_last_45year = data_frame[data_frame['Actual Date'] >= last_45_year][data_frame['Actual Date'] <= last_3year]
        sbls_bought_last_45year =  np.array(length_last_45year['sbl'])
        perm_last_45year = permits[permits['ISSUED'] >= last_45_year][permits['ISSUED'] <= last_3year]
        new_counter = 0
        # must double check if this data path file is in the server 
        df = pd.read_csv('/data/gentrification/TAX_DATA/Tax_Data/tax_data-' + str(last_date.year) + '.csv')
        try: 
            df_last3year = pd.read_csv('/data/gentrification/TAX_DATA/Tax_Data/tax_data-' + str(last_3year_bill_change) + '.csv')
        except:
            df_last3year = None 
        new_counter = 0
        counter +=  1 
        for sbl in unique_sbls:
            to_dataframe['SBL'].append(sbl)# SBL COLUMN
            to_dataframe['Timedelta'].append(timedelta_days)
            to_dataframe['Year'].append(dates.year)
            new_counter += 1 
            print( "OVERALL:" + str(counter / len(dates_six[1:]) * 100) + " |UNIQUE SBLS SECTION:"+ str(new_counter / len(unique_sbls) * 100))
            bought_this_year = True if sbl in sbls_bought_this_year else False
            to_dataframe['If Bought This Year'].append(bought_this_year) # IF HOUSE BOUGHT THIS YEAR COLUMN
            bought_first_three = True if sbl in sbls_bought_3month else False
            to_dataframe['Bought The First Three Months This Year'].append(bought_first_three) # IF HOUSE IS BOUGHT THE FIRST MONTHS THIS YEAR, THIS COLUMN WILL BE REMOVED
            bought_last_year = True if sbl in sbls_bought_last_year else False
            to_dataframe['If House is Bought Last Year'].append(bought_last_year) # IF HOUSE IS BOUGHT LAST YEAR
            bought_last_2year = True if sbl in sbls_bought_last_2year else False
            to_dataframe['If House is Bought Last 2 Years Ago'].append(bought_last_2year) # IF HOUSE IS BOUGHT LAST 2 Years Ago

            #--------------------------------------------------------------------------------
            neighbors_300 = w300.neighbors[sbl]
            neighbors_500 = w500.neighbors[sbl]    # LIST OF NEIGHBORS 
            neighbors_80 = w80.neighbors[sbl]
            n500 = data_frame[data_frame['sbl'].isin(neighbors_500)][data_frame['Actual Date'] <= last_date]
            n300 = data_frame[data_frame['sbl'].isin(neighbors_300)][data_frame['Actual Date'] <= last_date]
            n80 = data_frame[data_frame['sbl'].isin(neighbors_80)][data_frame['Actual Date'] <= last_date]


            # ---------------------------------------------------------------------------------------
            # Column for number of houses (80m) bought last year ago
            num_neighbors_bought_80_last_year = pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought Last Year'].append(num_neighbors_bought_80_last_year) 
            # Column for number of houses (80m) bought last 2 years ago
            num80_2yearsago = pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 2 Years Ago'].append(num80_2yearsago)
            # Column for number of houses (80m) bought last 3 years ago 
            num80_3yearsago = pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 3 Years Ago'].append(num80_3yearsago)
            # Column for number of houses (80m) bought last 4 years ago 
            num80_45yearsago =pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 4 Years Ago'].append(num80_45yearsago)


            #------------------------------------------------------------------------- # do duplicates here 
            # Column for number of houses bought (300m) bought last year ago 
            num_neighbors_bought_300_last_year = pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_300]).sum()  
            to_dataframe['Number of Houses (300m radius) Houses Bought Last Year'].append(num_neighbors_bought_300_last_year)
            # Column for number of houses bought (300m) * Not Going to add this column
            #num_neighbors_bought_300_3mnth =   pd.Series([ 1 for i in sbls_bought_3month if i in neighbors_300]).sum()     
            # Column for number of houses bought (300m) bought 2 years ago   
            num300_2yearsago =  pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_300]).sum() 
            to_dataframe['Number of Houses (300m radius) Houses Bought 2 years ago'].append(num300_2yearsago)
            # Column for number of houses bought (300m) bought 3 years ago
            num300_3yearsago =  pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_300]).sum()   
            to_dataframe['Number of Houses (300m radius) Houses Bought 3 years ago'].append(num300_3yearsago)
            # Column for number of houses bought (300m) bought 3 years ago 
            num300_45yearsago = pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_300]).sum()  
            to_dataframe['Number of Houses (300m radius) Houses Bought 4-5 years ago'].append(num300_45yearsago)
           # ------------------------------------------------------------------------------------------
            #num_neighbors_bought_500_6mnth =  pd.Series([ 1 for i in sbls_bought_this_year if i in neighbors_500]).sum()  
            #num_neighbors_bought_500_3mnth =  pd.Series([ 1 for i in sbls_bought_3month if i in neighbors_500]).sum()   
            # Column for number of houses bought (500m) last year ago 
            num500_1yearsago =  pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_500]).sum() 
            to_dataframe['Number of Houses (500m radius) Houses Bought last year ago'].append(num500_1yearsago)
            # Column for number of houses bought (500m) last 2 years ago 
            num500_2yearsago =  pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_500]).sum() 
            to_dataframe['Number of Houses (500m radius) Houses Bought 2 years ago'].append(num500_2yearsago)
            # Column for number of houses bought (500m) last 3 years ago 
            num500_3yearsago =  pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_500]).sum()  
            to_dataframe['Number of Houses (500m radius) Houses Bought 3 years ago'].append(num500_3yearsago)
            # Column for number of houses bought (500m) last 4-5 years ago 
            num500_45yearsago = pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_500]).sum()   
            to_dataframe['Number of Houses (500m radius) Houses Bought 4-5 years ago'].append(num500_45yearsago)
            #print(num_neighbors_bought_500_6mnth)   

            to_dataframe['Year | Month'].append(str(dates.year) + "|" + str(dates.month))
            #----------------PERMITS-----------------------------------------------------------------

            perm_500_lst_year =  perm_last_year[perm_last_year['sbl'].isin(neighbors_500)]
            perm_last_2year500 = perm_last_2year[perm_last_2year['sbl'].isin(neighbors_500)]
            perm_last_3year500 = perm_last_3year[perm_last_3year['sbl'].isin(neighbors_500)]
            perm_last_45year500 = perm_last_45year[perm_last_45year['sbl'].isin(neighbors_500)]



            #\\\\\\\\\\\\\\\
            perm_80_lst_year =  perm_last_year[perm_last_year['sbl'].isin(neighbors_80)]
            perm_last_2year80 = perm_last_2year[perm_last_2year['sbl'].isin(neighbors_80)]
            perm_last_3year80 = perm_last_3year[perm_last_3year['sbl'].isin(neighbors_80)]
            perm_last_45year80 = perm_last_45year[perm_last_45year['sbl'].isin(neighbors_80)]

            #-----------------------------------------------------------------------------------------

            clear_output(wait=False)
            ave_time_deltas = 0

            ave_price_sold_last_year_500 = n500[n500['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_300 = n300[n300['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_80 = n80[n80['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_500_change = n500[n500['Actual Date'] >= last_3year][n500['Actual Date'] <= last_2year]['Price'].mean()
            ave_price_sold_last_year_500_change = (ave_price_sold_last_year_500 - ave_price_sold_last_year_500_change) / ave_price_sold_last_year_500_change
            ave_bill_neighbors500 = df[df['SBL'].isin(neighbors_500)]['Principal'].astype(float).mean()
            sbl_last_year_500 = n500[n500['Actual Date'] >= last_year][['sbl','Price']].rename(columns = {'sbl':'SBL'}) # 500m radius bought last year 
            ratio = df[df['SBL'].isin(sbl_last_year_500['SBL'].unique())][['SBL','Principal']].set_index('SBL').to_dict()['Principal']
            sbl_last_year_500['princ'] = sbl_last_year_500['SBL'].map(ratio)
            num_foreclosures500 = len(n500[n500['Actual Date'] >= last_3year][n500['doc type'] == 'D1D']) 
            num_foreclosures300 = len(n300[n300['Actual Date'] >= last_3year][n300['doc type'] == 'D1D']) 
            num_foreclosures80 = len(n80[n80['Actual Date'] >= last_3year][n80['doc type'] == 'D1D']) 
            perms_3 = len(permits[permits['ISSUED'] >= last_3year][permits['ISSUED'] <= last_date][permits['sbl'].isin(neighbors_500)])
            try:
                rat = sbl_last_year_500[sbl_last_year_500['Price'] != 0][sbl_last_year_500['princ'] != 0] 
                mean_ratio = rat.apply(lambda x: x['Price'] / x['princ'], axis =1 ).apply(lambda i: i if isinstance(i,float) else np.nan).dropna().mean()

                if not isinstance(mean_ratio,float):
                    mean_ratio = 0
            except Exception as e:
                mean_ratio = 0 
                print(e)

            try:
                my_bill = df[df['SBL'] == sbl]['Principal'].astype(float)
                my_bill = my_bill.iloc[0] if len(my_bill) > 0 else 0  
                std_bill = my_bill - ave_bill_neighbors500
            except:
                std_bill =  0
            try:
                my_bill_last3 = df_last3year[df_last3year['SBL'] == sbl]['Principal'].astype(float) if df_last3year != None else None 
                my_bill_last3 =  my_bill_last3.iloc[0] if my_bill_last3 != None and len(my_bill_last3) > 0 else 0  
                bill_change= (my_bill - my_bill_last3) / my_bill_last3 if my_bill_last3 != -1 and my_bill != -1 else 0 
            except: 
                bill_change = 0

            ave_interest_near_me = df[df['SBL'].isin(neighbors_500)]['Pd Intr'].astype(float).mean()
            try:
                ave_time_deltas =  n500[n500['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas =  ave_time_deltas[ave_time_deltas  >= 0].mean() if len(ave_time_deltas) > 0else 0
                ave_time_deltas300 =  n300[n300['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas300 =  ave_time_deltas300[ave_time_deltas300  >= 0].mean() if len(ave_time_deltas300) > 0 else 0 
                ave_time_deltas80 =  n80[n80['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas80 =  ave_time_deltas80[ave_time_deltas80  >= 0].mean() if len(ave_time_deltas80) > 0 else 0

                ave_p500 = permits[permits['sbl'].isin(neighbors_500)][permits['ISSUED'] <= last_date]['ISSUED'].apply(lambda p: (last_date - p).days).mean()
                ave_p80 = permits[permits['sbl'].isin(neighbors_80)][permits['ISSUED'] <= last_date]['ISSUED'].apply(lambda p: (last_date - p).days).mean()
            except Exception as e:
                ave_time_deltas = 0
                ave_time_deltas300 = 0
                ave_time_deltas80 = 0
                print("OH NO")
                print(e)
                pass 
            try:
                house_length_homeownership = (last_date - data_frame[data_frame['sbl'] == sbl][data_frame['Actual Date'] <= last_date].iloc[-1]['Actual Date']).days
            except:
                house_length_homeownership = 0
            to_dataframe['Average length of homeownership (500m radius)'].append(ave_time_deltas)
            to_dataframe['length of homeownership (this house)'].append(house_length_homeownership)
            to_dataframe['Average bill of neighbors (500m radius) from last year'].append(ave_bill_neighbors500)
            to_dataframe['This house\'s bill last year'].append(my_bill)
            to_dataframe['Average interest of neighbors (500m radius) last year'].append(ave_interest_near_me)
            to_dataframe['Difference between SBL bill and neighbors (500m) last year'].append(std_bill)
            to_dataframe['Average length of homeownership (300m radius)'].append(ave_time_deltas300)
            to_dataframe['This sbl\'s bill percent change from last 3 years ago to last year'].append(bill_change)
            to_dataframe['Average length of homeownership (80m radius)'].append(ave_time_deltas80)
            to_dataframe['Average price sold of neighbors (500m radius) last year'].append(ave_price_sold_last_year_500)
            to_dataframe['Average price sold of neighbors (300m radius) last year'].append(ave_price_sold_last_year_300)
            to_dataframe['Average price sold of neighbors (80m radius) last year'].append(ave_price_sold_last_year_80)
            to_dataframe['Average percent change of house price transaction from last 3 years ago, to last year'].append(ave_price_sold_last_year_500_change)
            to_dataframe['Mean ratio of bill to house price transaction from last year (500m raidus)'].append(mean_ratio)
            to_dataframe['Number of permits last year in a (5oom) radius'].append(len(perm_500_lst_year))
            to_dataframe['Number of permits last 2 years ago in a (500m) radius'].append(len(perm_last_2year500))
            to_dataframe['Number of permits last 3 years ago in a (500m) radius'].append(len(perm_last_3year500))
            to_dataframe['Number of permits last 4-5 years ago in a (500m) radius'].append(len(perm_last_45year500))
            to_dataframe['Number of permits last year in a (80m) radius'].append(len(perm_80_lst_year))
            to_dataframe['Number of permits last 2 years ago in a (80m) radius'].append(len(perm_last_2year80))
            to_dataframe['Number of permits last 3 years ago in a (80m) radius'].append(len(perm_last_3year80))
            to_dataframe['Number of permits last 4-5 years ago in a (80m) radius'].append(len(perm_last_45year80))
            to_dataframe['Mean Value of work (Permits) in a (500m) radius last year'].append(perm_500_lst_year['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (500m) radius last 2 years ago'].append(perm_last_2year500['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (80m) radius last year'].append(perm_80_lst_year['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (80m) radius last 2 years ago'].append(perm_last_2year80['VALUE OF WORK'].mean())
            to_dataframe['Average length of time since the last permits issued in a (500m) radius'].append(ave_p500)
            to_dataframe['Average length of time since the last permits issued in a (80m) radius'].append(ave_p80)
            to_dataframe['Number of foreclosures in a (500m) radius, from last 3 years ago till last year'].append(num_foreclosures500)
            to_dataframe['Number of foreclosures in a (300m) radius, from last 3 years ago till last year'].append(num_foreclosures300)
            to_dataframe['Number of foreclosures in a (80m) radius, from last 3 years ago till last year'].append(num_foreclosures80)
            to_dataframe['Number of Permits from last 3 years ago, to last year, in a (500m) radius'].append(perms_3)
            try:
                neighbor_name  = neigh[sbl]
            except:
                neighbor_name = 'UNKNOWN' 

            # one hot encode neighborhoods 
            for neighborhood in list_of_neighborhoods:
                if neighborhood == neighborhood and neighborhood != neighbor_name:
                    to_dataframe[neighborhood].append(0)
                else: 
                    to_dataframe[neighbor_name].append(1) 
            # this way its just easier to name columns
        last_date = dates
    return pd.DataFrame(to_dataframe)  

In [34]:
def link_locations(data, open_data_df):
    neigh_map = open_data_df[['PRINT KEY','NEIGHBORHOOD']].set_index('PRINT KEY').to_dict()['NEIGHBORHOOD']
    location_map = open_data_df[['PRINT KEY','LOCATION']].set_index('PRINT KEY').to_dict()['LOCATION']
    data['Neighborhood'] = data['sbl'].map(neigh_map)
    data['Location'] = data['sbl'].map(location_map)
    return data

In [58]:
def filter_apartments(data,path):
    path_property_information = os.path.join(path, 'SCRAPED_DATA_property_information' ) # will chnage to 'SCRAPED_DATA_property_information'
    path_owner_history = os.path.join(path, 'SCRAPED_DATA_owner_history') 
    lists = os.listdir(path_property_information)
    list_of_failed_files = []
    counter = 0
    lists_of_residential_sbls = []
    list_of_anomolies = []
    for i in lists:
        try:
            print(counter / len(lists) * 100)
            property_information_sbl_path  = os.path.join(path_property_information,i)
            owner_history_sbl_path = os.path.join(path_owner_history,i)
            f_front_htmls =codecs.open(property_information_sbl_path, 'r')
            f_owner_history_htmls =codecs.open(owner_history_sbl_path, 'r')
            soup_front_htmls = BeautifulSoup(str(f_front_htmls.read()),'html.parser')
            soup_owner_history_htmls = BeautifulSoup(str(f_owner_history_htmls.read()),'html.parser')

            df_front_htmls = pd.read_html(property_information_sbl_path)[0]
            df_owner_history_htmls = pd.read_html(owner_history_sbl_path)[0]
            num_of_res =  df_front_htmls[1].apply(lambda x: 'RES' in x.split() if x == x else False).sum() 
            if num_of_res > 0 and num_of_res < 2:
                lists_of_residential_sbls.append(i.split('=')[1][:-5].replace('#','.').replace('+','/'))
            elif num_of_res == 0: # does not contain any residential
                continue
            else:
             
                list_of_anomolies.append(df_front_htmls[1])
            
            counter += 1 
            clear_output()
        except Exception as p:
            list_of_failed_files.append(i)
    
    lists_of_residential_sbls = np.array(lists_of_residential_sbls)
    return data[data['sbl'].isin(lists_of_residential_sbls)], list_of_failed_files, list_of_anomolies 

In [71]:
def data_frame_construction_for_rf(dataframe):
    dataframe = dataframe.copy()
    dataframe['Price'] = dataframe['consideration'].apply(lambda i: float(i.replace('$','').replace(',','')) if i == i else np.nan )
    dataframe =  dataframe[~dataframe['Actual Date'].isna()]
    dataframe = dataframe[dataframe['Price']  > 1]
    dataframe_more_than_one_doc_type = dataframe[dataframe['more than one doc type']]
    index_of_single_doc =  dataframe_more_than_one_doc_type.drop_duplicates(['sbl','Actual Date']).index
    idx_to_remove = dataframe_more_than_one_doc_type[~dataframe_more_than_one_doc_type.index.isin(index_of_single_doc)].index 
    dataframe = dataframe.drop(idx_to_remove)
    dataframe = dataframe[~dataframe['Location'].isnull()]
    dataframe= dataframe.sort_values(by = 'Actual Date')
    dataframe['Actual Date'] = pd.to_datetime(dataframe['Actual Date'])
    return dataframe 

In [67]:
def batch_geoencode_missing_location(dataframe,path):
    dataframe = dataframe.copy()
    oarsystem_data_path = os.path.join(path,'buffalo_oarsystem_CSVs/combined_tax_data.csv')
    oarsystem_df = pd.read_csv(os.path.join(oarsystem_data_path)).rename(columns = {'SBL':'sbl'})
     # note this was scraped before hand; the aim is to take ADDRESS here 
    merged_df = dataframe.merge(oarsystem_df[['sbl','ADDRESS']].drop_duplicates(subset = ['sbl']), on = ['sbl'])
    missing_location_addresses = merged_df[merged_df['Location'].isna()]['ADDRESS']
    df_to_geoencode = missing_location_addresses.str.split(',', expand=True)
    df_to_geoencode.insert(2,'3','NY')
    if os.path.exists('batch_file.csv'):
        print("batch files have been added!")
    else:
        df_to_geoencode.to_csv('batch_file.csv')
    if os.path.exists('geocoderesult.csv'):
        print("Locations have been geocoded!")
    else:
        !curl --form addressFile=@batch_file.csv --form benchmark=9 https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output geocoderesult.csv
    result = pd.read_csv('geocoderesult.csv')
    result = result.reset_index()[['level_0','Unnamed: 0']].set_index('level_0')
    result['Unnamed: 0'] = result['Unnamed: 0'].apply(lambda x: '(' + x + ')' if x== x else np.nan)
    indexes  = result.index
    values = result.values.flatten().tolist()
    dataframe.loc[indexes, 'Location'] = values
    return dataframe
 
    

In [ ]:
def fix_dates(dataframe):
    '''
    for every anomoly, we try to replace them with the record given within the public records website 
    '''
    dataframe = fixed_dataframe.copy()
    index =dataframe[dataframe['date(search parcel)'].isna()].index
    dataframe['Actual Date'] = dataframe['date(search parcel)']
    dataframe.loc[index,'Actual Date'] = dataframe.loc[index,'date(public records)']
    dates = pd.to_datetime(dataframe['date(search parcel)'],errors = 'coerce')
    index_greater_2020 = dataframe[dates.apply(lambda x: True if x.year > 2020 else False)].index # index of anomolies
    dataframe.loc[index_greater_2020,'Actual Date'] = dataframe.loc[index_greater_2020, 'date(public records)']
    dataframe['Actual Date'] = pd.to_datetime(dataframe['Actual Date'], errors = 'coerce')
    return dataframe 

In [ ]:
def add_multiple_doc_types_column(dataframe):
    dataframe = dataframe.copy()
    group_by_doc_type = dataframe.groupby(['sbl','Actual Date'])['doc type'].count() 
    anomolies = group_by_doc_type[group_by_doc_type > 1].reset_index()[['sbl','Actual Date']]
    anomolies['more than one doc type'] = True 
    dataframe = dataframe.merge(anomolies, on = ['sbl','Actual Date'],how = 'left',)
    dataframe['more than one doc type'] = dataframe['more than one doc type'].fillna(False)
    return dataframe 

In [2]:
def random_classifier_data_pre_processing2(data_frame,open_dat):

    # permits data acess 
    
    permits_data =  pd.read_csv('https://data.buffalony.gov/api/views/9p2d-f3yt/rows.csv?accessType=DOWNLOAD', low_memory= False)
    
    permits_data['ISSUED'] = pd.to_datetime(permits_data['ISSUED'])
    permits = open_dat[['SBL','PRINT KEY',]].merge(permits_data, on = 'SBL', how = 'right')
    permits = permits.rename(columns = {'PRINT KEY':'sbl'}).drop('SBL',axis =1 )
    
    # -----------------------------------------------------------------
    # -------  Weights and data initialization ------------------------
    data_frame = data_frame.copy()
    to_dataframe = defaultdict(list)
    counter = 0
    list_of_neighborhoods = data_frame['Neighborhood'].unique()[:-1]
    w500, w300, w80, data_frame, unique_sbls = weights_initialization(data_frame, 'Location')
    dates_six = pd.date_range(start="1993-12-01",end="2021-12-01", freq='A-DEC')[1:]
    last_date = dates_six[0]
    
    # -------- loop feature collection --------------------------------
    
    for dates in dates_six[1:]: # we start from the second to the first date
        timedelta_days = (dates - parser.parse('1854-10-01')).days
        three_months = last_date + timedelta(90)
        last_year = last_date - timedelta(365)
        last_2year = last_date - timedelta(730)
        last_3year = last_date - timedelta(1095)
        last_45_year = last_date - timedelta(1825)
        #------
        length = data_frame[data_frame['Actual Date'] > last_date][data_frame['Actual Date'] <= dates]
        sbls_bought_this_year =  np.array(length['sbl']) # drop duplicates dates here just to be safe 
        #----
        length_3month = data_frame[data_frame['Actual Date'] > last_date][data_frame['Actual Date'] <= three_months]
        sbls_bought_3month =  np.array(length_3month['sbl'])
        #-----
        length_last_year = data_frame[data_frame['Actual Date'] >= last_year][data_frame['Actual Date'] <= last_date]
        sbls_bought_last_year =  set(np.array(length_last_year['sbl']))
        perm_last_year = permits[permits['ISSUED'] >= last_year][permits['ISSUED'] <= last_date]
        #----
        length_last_2year = data_frame[data_frame['Actual Date'] >= last_2year][data_frame['Actual Date'] <= last_year]
        sbls_bought_last_2year =  np.array(length_last_2year['sbl'])
        perm_last_2year = permits[permits['ISSUED'] >= last_2year][permits['ISSUED'] <= last_year]
        # -- 
        length_last_3year = data_frame[data_frame['Actual Date'] >= last_3year][data_frame['Actual Date'] <= last_2year]
        sbls_bought_last_3year =  np.array(length_last_3year['sbl'])
        perm_last_3year = permits[permits['ISSUED'] >= last_3year][permits['ISSUED'] <= last_2year]
        # -- 
        length_last_45year = data_frame[data_frame['Actual Date'] >= last_45_year][data_frame['Actual Date'] <= last_3year]
        sbls_bought_last_45year =  np.array(length_last_45year['sbl'])
        perm_last_45year = permits[permits['ISSUED'] >= last_45_year][permits['ISSUED'] <= last_3year]
        new_counter = 0
        # must double check if this data path file is in the server 
        df = pd.read_csv('/data/gentrification/TAX_DATA/Tax_Data/tax_data-' + str(last_date.year) + '.csv')
        try: 
            df_last3year = pd.read_csv('/data/gentrification/TAX_DATA/Tax_Data/tax_data-' + str(last_3year_bill_change) + '.csv')
        except:
            df_last3year = None 
        new_counter = 0
        counter +=  1 
        for sbl in unique_sbls:
            to_dataframe['SBL'].append(sbl)# SBL COLUMN
            to_dataframe['Timedelta'].append(timedelta_days)
            to_dataframe['Year'].append(dates.year)
            new_counter += 1 
            print( "OVERALL:" + str(counter / len(dates_six[1:]) * 100) + " |UNIQUE SBLS SECTION:"+ str(new_counter / len(unique_sbls) * 100))
            try:
                bought_this_year = (data_frame[data_frame['sbl'] == sbl][data_frame['Actual Date'] > last_date].iloc[0]['Actual Date'] -last_date).days
            except:
                bought_this_year = 0
            # bought_this_year = True if sbl in sbls_bought_this_year else False
            to_dataframe['If Bought This Year'].append(bought_this_year) # IF HOUSE BOUGHT THIS YEAR COLUMN
            bought_first_three = True if sbl in sbls_bought_3month else False
            to_dataframe['Bought The First Three Months This Year'].append(bought_first_three) # IF HOUSE IS BOUGHT THE FIRST MONTHS THIS YEAR, THIS COLUMN WILL BE REMOVED
            bought_last_year = True if sbl in sbls_bought_last_year else False
            to_dataframe['If House is Bought Last Year'].append(bought_last_year) # IF HOUSE IS BOUGHT LAST YEAR
            bought_last_2year = True if sbl in sbls_bought_last_2year else False
            to_dataframe['If House is Bought Last 2 Years Ago'].append(bought_last_2year) # IF HOUSE IS BOUGHT LAST 2 Years Ago

            #--------------------------------------------------------------------------------
            neighbors_300 = w300.neighbors[sbl]
            neighbors_500 = w500.neighbors[sbl]    # LIST OF NEIGHBORS 
            neighbors_80 = w80.neighbors[sbl]
            n500 = data_frame[data_frame['sbl'].isin(neighbors_500)][data_frame['Actual Date'] <= last_date]
            n300 = data_frame[data_frame['sbl'].isin(neighbors_300)][data_frame['Actual Date'] <= last_date]
            n80 = data_frame[data_frame['sbl'].isin(neighbors_80)][data_frame['Actual Date'] <= last_date]


            # ---------------------------------------------------------------------------------------
            # Column for number of houses (80m) bought last year ago
            num_neighbors_bought_80_last_year = pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought Last Year'].append(num_neighbors_bought_80_last_year) 
            # Column for number of houses (80m) bought last 2 years ago
            num80_2yearsago = pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 2 Years Ago'].append(num80_2yearsago)
            # Column for number of houses (80m) bought last 3 years ago 
            num80_3yearsago = pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 3 Years Ago'].append(num80_3yearsago)
            # Column for number of houses (80m) bought last 4 years ago 
            num80_45yearsago =pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 4 Years Ago'].append(num80_45yearsago)


            #------------------------------------------------------------------------- # do duplicates here 
            # Column for number of houses bought (300m) bought last year ago 
            num_neighbors_bought_300_last_year = pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_300]).sum()  
            to_dataframe['Number of Houses (300m radius) Houses Bought Last Year'].append(num_neighbors_bought_300_last_year)
            # Column for number of houses bought (300m) * Not Going to add this column
            #num_neighbors_bought_300_3mnth =   pd.Series([ 1 for i in sbls_bought_3month if i in neighbors_300]).sum()     
            # Column for number of houses bought (300m) bought 2 years ago   
            num300_2yearsago =  pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_300]).sum() 
            to_dataframe['Number of Houses (300m radius) Houses Bought 2 years ago'].append(num300_2yearsago)
            # Column for number of houses bought (300m) bought 3 years ago
            num300_3yearsago =  pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_300]).sum()   
            to_dataframe['Number of Houses (300m radius) Houses Bought 3 years ago'].append(num300_3yearsago)
            # Column for number of houses bought (300m) bought 3 years ago 
            num300_45yearsago = pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_300]).sum()  
            to_dataframe['Number of Houses (300m radius) Houses Bought 4-5 years ago'].append(num300_45yearsago)
           # ------------------------------------------------------------------------------------------
            #num_neighbors_bought_500_6mnth =  pd.Series([ 1 for i in sbls_bought_this_year if i in neighbors_500]).sum()  
            #num_neighbors_bought_500_3mnth =  pd.Series([ 1 for i in sbls_bought_3month if i in neighbors_500]).sum()   
            # Column for number of houses bought (500m) last year ago 
            num500_1yearsago =  pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_500]).sum() 
            to_dataframe['Number of Houses (500m radius) Houses Bought last year ago'].append(num500_1yearsago)
            # Column for number of houses bought (500m) last 2 years ago 
            num500_2yearsago =  pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_500]).sum() 
            to_dataframe['Number of Houses (500m radius) Houses Bought 2 years ago'].append(num500_2yearsago)
            # Column for number of houses bought (500m) last 3 years ago 
            num500_3yearsago =  pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_500]).sum()  
            to_dataframe['Number of Houses (500m radius) Houses Bought 3 years ago'].append(num500_3yearsago)
            # Column for number of houses bought (500m) last 4-5 years ago 
            num500_45yearsago = pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_500]).sum()   
            to_dataframe['Number of Houses (500m radius) Houses Bought 4-5 years ago'].append(num500_45yearsago)
            #print(num_neighbors_bought_500_6mnth)   

            to_dataframe['Year | Month'].append(str(dates.year) + "|" + str(dates.month))
            #----------------PERMITS-----------------------------------------------------------------

            perm_500_lst_year =  perm_last_year[perm_last_year['sbl'].isin(neighbors_500)]
            perm_last_2year500 = perm_last_2year[perm_last_2year['sbl'].isin(neighbors_500)]
            perm_last_3year500 = perm_last_3year[perm_last_3year['sbl'].isin(neighbors_500)]
            perm_last_45year500 = perm_last_45year[perm_last_45year['sbl'].isin(neighbors_500)]



            #\\\\\\\\\\\\\\\
            perm_80_lst_year =  perm_last_year[perm_last_year['sbl'].isin(neighbors_80)]
            perm_last_2year80 = perm_last_2year[perm_last_2year['sbl'].isin(neighbors_80)]
            perm_last_3year80 = perm_last_3year[perm_last_3year['sbl'].isin(neighbors_80)]
            perm_last_45year80 = perm_last_45year[perm_last_45year['sbl'].isin(neighbors_80)]

            #-----------------------------------------------------------------------------------------

            clear_output(wait=False)
            ave_time_deltas = 0

            ave_price_sold_last_year_500 = n500[n500['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_300 = n300[n300['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_80 = n80[n80['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_500_change = n500[n500['Actual Date'] >= last_3year][n500['Actual Date'] <= last_2year]['Price'].mean()
            ave_price_sold_last_year_500_change = (ave_price_sold_last_year_500 - ave_price_sold_last_year_500_change) / ave_price_sold_last_year_500_change
            ave_bill_neighbors500 = df[df['SBL'].isin(neighbors_500)]['Principal'].astype(float).mean()
            sbl_last_year_500 = n500[n500['Actual Date'] >= last_year][['sbl','Price']].rename(columns = {'sbl':'SBL'}) # 500m radius bought last year 
            ratio = df[df['SBL'].isin(sbl_last_year_500['SBL'].unique())][['SBL','Principal']].set_index('SBL').to_dict()['Principal']
            sbl_last_year_500['princ'] = sbl_last_year_500['SBL'].map(ratio)
            num_foreclosures500 = len(n500[n500['Actual Date'] >= last_3year][n500['doc type'] == 'D1D']) 
            num_foreclosures300 = len(n300[n300['Actual Date'] >= last_3year][n300['doc type'] == 'D1D']) 
            num_foreclosures80 = len(n80[n80['Actual Date'] >= last_3year][n80['doc type'] == 'D1D']) 
            perms_3 = len(permits[permits['ISSUED'] >= last_3year][permits['ISSUED'] <= last_date][permits['sbl'].isin(neighbors_500)])
            try:
                rat = sbl_last_year_500[sbl_last_year_500['Price'] != 0][sbl_last_year_500['princ'] != 0] 
                mean_ratio = rat.apply(lambda x: x['Price'] / x['princ'], axis =1 ).apply(lambda i: i if isinstance(i,float) else np.nan).dropna().mean()

                if not isinstance(mean_ratio,float):
                    mean_ratio = 0
            except Exception as e:
                mean_ratio = 0 
                print(e)

            try:
                my_bill = df[df['SBL'] == sbl]['Principal'].astype(float)
                my_bill = my_bill.iloc[0] if len(my_bill) > 0 else 0  
                std_bill = my_bill - ave_bill_neighbors500
            except:
                std_bill =  0
            try:
                my_bill_last3 = df_last3year[df_last3year['SBL'] == sbl]['Principal'].astype(float) if df_last3year != None else None 
                my_bill_last3 =  my_bill_last3.iloc[0] if my_bill_last3 != None and len(my_bill_last3) > 0 else 0  
                bill_change= (my_bill - my_bill_last3) / my_bill_last3 if my_bill_last3 != -1 and my_bill != -1 else 0 
            except: 
                bill_change = 0

            ave_interest_near_me = df[df['SBL'].isin(neighbors_500)]['Pd Intr'].astype(float).mean()
            try:
                ave_time_deltas =  n500[n500['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas =  ave_time_deltas[ave_time_deltas  >= 0].mean() if len(ave_time_deltas) > 0else 0
                ave_time_deltas300 =  n300[n300['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas300 =  ave_time_deltas300[ave_time_deltas300  >= 0].mean() if len(ave_time_deltas300) > 0 else 0 
                ave_time_deltas80 =  n80[n80['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas80 =  ave_time_deltas80[ave_time_deltas80  >= 0].mean() if len(ave_time_deltas80) > 0 else 0

                ave_p500 = permits[permits['sbl'].isin(neighbors_500)][permits['ISSUED'] <= last_date]['ISSUED'].apply(lambda p: (last_date - p).days).mean()
                ave_p80 = permits[permits['sbl'].isin(neighbors_80)][permits['ISSUED'] <= last_date]['ISSUED'].apply(lambda p: (last_date - p).days).mean()
            except Exception as e:
                ave_time_deltas = 0
                ave_time_deltas300 = 0
                ave_time_deltas80 = 0
                print("OH NO")
                print(e)
                pass 
            try:
                house_length_homeownership = (last_date - data_frame[data_frame['sbl'] == sbl][data_frame['Actual Date'] <= last_date].iloc[-1]['Actual Date']).days
            except:
                house_length_homeownership = 0
            to_dataframe['Average length of homeownership (500m radius)'].append(ave_time_deltas)
            to_dataframe['length of homeownership (this house)'].append(house_length_homeownership)
            to_dataframe['Average bill of neighbors (500m radius) from last year'].append(ave_bill_neighbors500)
            to_dataframe['This house\'s bill last year'].append(my_bill)
            to_dataframe['Average interest of neighbors (500m radius) last year'].append(ave_interest_near_me)
            to_dataframe['Difference between SBL bill and neighbors (500m) last year'].append(std_bill)
            to_dataframe['Average length of homeownership (300m radius)'].append(ave_time_deltas300)
            to_dataframe['This sbl\'s bill percent change from last 3 years ago to last year'].append(bill_change)
            to_dataframe['Average length of homeownership (80m radius)'].append(ave_time_deltas80)
            to_dataframe['Average price sold of neighbors (500m radius) last year'].append(ave_price_sold_last_year_500)
            to_dataframe['Average price sold of neighbors (300m radius) last year'].append(ave_price_sold_last_year_300)
            to_dataframe['Average price sold of neighbors (80m radius) last year'].append(ave_price_sold_last_year_80)
            to_dataframe['Average percent change of house price transaction from last 3 years ago, to last year'].append(ave_price_sold_last_year_500_change)
            to_dataframe['Mean ratio of bill to house price transaction from last year (500m raidus)'].append(mean_ratio)
            to_dataframe['Number of permits last year in a (5oom) radius'].append(len(perm_500_lst_year))
            to_dataframe['Number of permits last 2 years ago in a (500m) radius'].append(len(perm_last_2year500))
            to_dataframe['Number of permits last 3 years ago in a (500m) radius'].append(len(perm_last_3year500))
            to_dataframe['Number of permits last 4-5 years ago in a (500m) radius'].append(len(perm_last_45year500))
            to_dataframe['Number of permits last year in a (80m) radius'].append(len(perm_80_lst_year))
            to_dataframe['Number of permits last 2 years ago in a (80m) radius'].append(len(perm_last_2year80))
            to_dataframe['Number of permits last 3 years ago in a (80m) radius'].append(len(perm_last_3year80))
            to_dataframe['Number of permits last 4-5 years ago in a (80m) radius'].append(len(perm_last_45year80))
            to_dataframe['Mean Value of work (Permits) in a (500m) radius last year'].append(perm_500_lst_year['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (500m) radius last 2 years ago'].append(perm_last_2year500['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (80m) radius last year'].append(perm_80_lst_year['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (80m) radius last 2 years ago'].append(perm_last_2year80['VALUE OF WORK'].mean())
            to_dataframe['Average length of time since the last permits issued in a (500m) radius'].append(ave_p500)
            to_dataframe['Average length of time since the last permits issued in a (80m) radius'].append(ave_p80)
            to_dataframe['Number of foreclosures in a (500m) radius, from last 3 years ago till last year'].append(num_foreclosures500)
            to_dataframe['Number of foreclosures in a (300m) radius, from last 3 years ago till last year'].append(num_foreclosures300)
            to_dataframe['Number of foreclosures in a (80m) radius, from last 3 years ago till last year'].append(num_foreclosures80)
            to_dataframe['Number of Permits from last 3 years ago, to last year, in a (500m) radius'].append(perms_3)
            try:
                neighbor_name  = neigh[sbl]
            except:
                neighbor_name = 'UNKNOWN' 

            # one hot encode neighborhoods 
            for neighborhood in list_of_neighborhoods:
                if neighborhood == neighborhood and neighborhood != neighbor_name:
                    to_dataframe[neighborhood].append(0)
                else: 
                    to_dataframe[neighbor_name].append(1) 
            # this way its just easier to name columns
        last_date = dates
    return pd.DataFrame(to_dataframe)  

In [ ]:
def random_classifier_data_pre_processing3(data_frame,open_dat):
    # i can do it before years
    # permits data acess 
    
    permits_data =  pd.read_csv('https://data.buffalony.gov/api/views/9p2d-f3yt/rows.csv?accessType=DOWNLOAD', low_memory= False)
    
    permits_data['ISSUED'] = pd.to_datetime(permits_data['ISSUED'])
    permits = open_dat[['SBL','PRINT KEY',]].merge(permits_data, on = 'SBL', how = 'right')
    permits = permits.rename(columns = {'PRINT KEY':'sbl'}).drop('SBL',axis =1 )
    
    # -----------------------------------------------------------------
    # -------  Weights and data initialization ------------------------
    data_frame = data_frame.copy()
    to_dataframe = defaultdict(list)
    counter = 0
    list_of_neighborhoods = data_frame['Neighborhood'].unique()[:-1]
    w500, w300, w80, data_frame, unique_sbls = weights_initialization(data_frame, 'Location')
    dates_six = pd.date_range(start="1993-12-01",end="2021-12-01", freq='A-DEC')[1:]
    last_date = dates_six[0]
    
    # -------- loop feature collection --------------------------------
    
    for dates in dates_six[1:]: # we start from the second to the first date
        timedelta_days = (dates - parser.parse('1854-10-01')).days
        three_months = last_date + timedelta(90)
        last_year = last_date - timedelta(365)
        last_2year = last_date - timedelta(730)
        last_3year = last_date - timedelta(1095)
        last_45_year = last_date - timedelta(1825)
        #------
        length = data_frame[data_frame['Actual Date'] > last_date][data_frame['Actual Date'] <= dates]
        sbls_bought_this_year =  np.array(length['sbl']) # drop duplicates dates here just to be safe 
        #----
        length_3month = data_frame[data_frame['Actual Date'] > last_date][data_frame['Actual Date'] <= three_months]
        sbls_bought_3month =  np.array(length_3month['sbl'])
        #-----
        length_last_year = data_frame[data_frame['Actual Date'] >= last_year][data_frame['Actual Date'] <= last_date]
        sbls_bought_last_year =  set(np.array(length_last_year['sbl']))
        perm_last_year = permits[permits['ISSUED'] >= last_year][permits['ISSUED'] <= last_date]
        #----
        length_last_2year = data_frame[data_frame['Actual Date'] >= last_2year][data_frame['Actual Date'] <= last_year]
        sbls_bought_last_2year =  np.array(length_last_2year['sbl'])
        perm_last_2year = permits[permits['ISSUED'] >= last_2year][permits['ISSUED'] <= last_year]
        # -- 
        length_last_3year = data_frame[data_frame['Actual Date'] >= last_3year][data_frame['Actual Date'] <= last_2year]
        sbls_bought_last_3year =  np.array(length_last_3year['sbl'])
        perm_last_3year = permits[permits['ISSUED'] >= last_3year][permits['ISSUED'] <= last_2year]
        # -- 
        length_last_45year = data_frame[data_frame['Actual Date'] >= last_45_year][data_frame['Actual Date'] <= last_3year]
        sbls_bought_last_45year =  np.array(length_last_45year['sbl'])
        perm_last_45year = permits[permits['ISSUED'] >= last_45_year][permits['ISSUED'] <= last_3year]
        new_counter = 0
        # must double check if this data path file is in the server 
        df = pd.read_csv('/data/gentrification/TAX_DATA/Tax_Data/tax_data-' + str(last_date.year) + '.csv')
        try: 
            df_last3year = pd.read_csv('/data/gentrification/TAX_DATA/Tax_Data/tax_data-' + str(last_3year_bill_change) + '.csv')
        except:
            df_last3year = None 
        new_counter = 0
        counter +=  1 
        for sbl in unique_sbls:
            to_dataframe['SBL'].append(sbl)# SBL COLUMN
            to_dataframe['Timedelta'].append(timedelta_days)
            to_dataframe['Year'].append(dates.year)
            new_counter += 1 
            print( "OVERALL:" + str(counter / len(dates_six[1:]) * 100) + " |UNIQUE SBLS SECTION:"+ str(new_counter / len(unique_sbls) * 100))
            try:
                bought_this_year = (data_frame[data_frame['sbl'] == sbl][data_frame['Actual Date'] > last_date].iloc[0]['Actual Date'] -last_date).days
            except:
                bought_this_year = 0
            # bought_this_year = True if sbl in sbls_bought_this_year else False
            to_dataframe['If Bought This Year'].append(bought_this_year) # IF HOUSE BOUGHT THIS YEAR COLUMN
            bought_first_three = True if sbl in sbls_bought_3month else False
            to_dataframe['Bought The First Three Months This Year'].append(bought_first_three) # IF HOUSE IS BOUGHT THE FIRST MONTHS THIS YEAR, THIS COLUMN WILL BE REMOVED
            bought_last_year = True if sbl in sbls_bought_last_year else False
            to_dataframe['If House is Bought Last Year'].append(bought_last_year) # IF HOUSE IS BOUGHT LAST YEAR
            bought_last_2year = True if sbl in sbls_bought_last_2year else False
            to_dataframe['If House is Bought Last 2 Years Ago'].append(bought_last_2year) # IF HOUSE IS BOUGHT LAST 2 Years Ago

            #--------------------------------------------------------------------------------
            neighbors_300 = w300.neighbors[sbl]
            neighbors_500 = w500.neighbors[sbl]    # LIST OF NEIGHBORS 
            neighbors_80 = w80.neighbors[sbl]
            n500 = data_frame[data_frame['sbl'].isin(neighbors_500)][data_frame['Actual Date'] <= last_date]
            n300 = data_frame[data_frame['sbl'].isin(neighbors_300)][data_frame['Actual Date'] <= last_date]
            n80 = data_frame[data_frame['sbl'].isin(neighbors_80)][data_frame['Actual Date'] <= last_date]


            # ---------------------------------------------------------------------------------------
            # Column for number of houses (80m) bought last year ago
            num_neighbors_bought_80_last_year = pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought Last Year'].append(num_neighbors_bought_80_last_year) 
            # Column for number of houses (80m) bought last 2 years ago
            num80_2yearsago = pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 2 Years Ago'].append(num80_2yearsago)
            # Column for number of houses (80m) bought last 3 years ago 
            num80_3yearsago = pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 3 Years Ago'].append(num80_3yearsago)
            # Column for number of houses (80m) bought last 4 years ago 
            num80_45yearsago =pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_80]).sum()
            to_dataframe['Number of Houses (80m radius) Houses Bought 4 Years Ago'].append(num80_45yearsago)


            #------------------------------------------------------------------------- # do duplicates here 
            # Column for number of houses bought (300m) bought last year ago 
            num_neighbors_bought_300_last_year = pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_300]).sum()  
            to_dataframe['Number of Houses (300m radius) Houses Bought Last Year'].append(num_neighbors_bought_300_last_year)
            # Column for number of houses bought (300m) * Not Going to add this column
            #num_neighbors_bought_300_3mnth =   pd.Series([ 1 for i in sbls_bought_3month if i in neighbors_300]).sum()     
            # Column for number of houses bought (300m) bought 2 years ago   
            num300_2yearsago =  pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_300]).sum() 
            to_dataframe['Number of Houses (300m radius) Houses Bought 2 years ago'].append(num300_2yearsago)
            # Column for number of houses bought (300m) bought 3 years ago
            num300_3yearsago =  pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_300]).sum()   
            to_dataframe['Number of Houses (300m radius) Houses Bought 3 years ago'].append(num300_3yearsago)
            # Column for number of houses bought (300m) bought 3 years ago 
            num300_45yearsago = pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_300]).sum()  
            to_dataframe['Number of Houses (300m radius) Houses Bought 4-5 years ago'].append(num300_45yearsago)
           # ------------------------------------------------------------------------------------------
            #num_neighbors_bought_500_6mnth =  pd.Series([ 1 for i in sbls_bought_this_year if i in neighbors_500]).sum()  
            #num_neighbors_bought_500_3mnth =  pd.Series([ 1 for i in sbls_bought_3month if i in neighbors_500]).sum()   
            # Column for number of houses bought (500m) last year ago 
            num500_1yearsago =  pd.Series([ 1 for i in sbls_bought_last_year if i in neighbors_500]).sum() 
            to_dataframe['Number of Houses (500m radius) Houses Bought last year ago'].append(num500_1yearsago)
            # Column for number of houses bought (500m) last 2 years ago 
            num500_2yearsago =  pd.Series([ 1 for i in sbls_bought_last_2year if i in neighbors_500]).sum() 
            to_dataframe['Number of Houses (500m radius) Houses Bought 2 years ago'].append(num500_2yearsago)
            # Column for number of houses bought (500m) last 3 years ago 
            num500_3yearsago =  pd.Series([ 1 for i in sbls_bought_last_3year if i in neighbors_500]).sum()  
            to_dataframe['Number of Houses (500m radius) Houses Bought 3 years ago'].append(num500_3yearsago)
            # Column for number of houses bought (500m) last 4-5 years ago 
            num500_45yearsago = pd.Series([ 1 for i in sbls_bought_last_45year if i in neighbors_500]).sum()   
            to_dataframe['Number of Houses (500m radius) Houses Bought 4-5 years ago'].append(num500_45yearsago)
            #print(num_neighbors_bought_500_6mnth)   

            to_dataframe['Year | Month'].append(str(dates.year) + "|" + str(dates.month))
            #----------------PERMITS-----------------------------------------------------------------

            perm_500_lst_year =  perm_last_year[perm_last_year['sbl'].isin(neighbors_500)]
            perm_last_2year500 = perm_last_2year[perm_last_2year['sbl'].isin(neighbors_500)]
            perm_last_3year500 = perm_last_3year[perm_last_3year['sbl'].isin(neighbors_500)]
            perm_last_45year500 = perm_last_45year[perm_last_45year['sbl'].isin(neighbors_500)]



            #\\\\\\\\\\\\\\\
            perm_80_lst_year =  perm_last_year[perm_last_year['sbl'].isin(neighbors_80)]
            perm_last_2year80 = perm_last_2year[perm_last_2year['sbl'].isin(neighbors_80)]
            perm_last_3year80 = perm_last_3year[perm_last_3year['sbl'].isin(neighbors_80)]
            perm_last_45year80 = perm_last_45year[perm_last_45year['sbl'].isin(neighbors_80)]

            #-----------------------------------------------------------------------------------------

            clear_output(wait=False)
            ave_time_deltas = 0

            ave_price_sold_last_year_500 = n500[n500['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_300 = n300[n300['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_80 = n80[n80['Actual Date'] >= last_year]['Price'].dropna().mean()
            ave_price_sold_last_year_500_change = n500[n500['Actual Date'] >= last_3year][n500['Actual Date'] <= last_2year]['Price'].mean()
            ave_price_sold_last_year_500_change = (ave_price_sold_last_year_500 - ave_price_sold_last_year_500_change) / ave_price_sold_last_year_500_change
            ave_bill_neighbors500 = df[df['SBL'].isin(neighbors_500)]['Principal'].astype(float).mean()
            sbl_last_year_500 = n500[n500['Actual Date'] >= last_year][['sbl','Price']].rename(columns = {'sbl':'SBL'}) # 500m radius bought last year 
            ratio = df[df['SBL'].isin(sbl_last_year_500['SBL'].unique())][['SBL','Principal']].set_index('SBL').to_dict()['Principal']
            sbl_last_year_500['princ'] = sbl_last_year_500['SBL'].map(ratio)
            num_foreclosures500 = len(n500[n500['Actual Date'] >= last_3year][n500['doc type'] == 'D1D']) 
            num_foreclosures300 = len(n300[n300['Actual Date'] >= last_3year][n300['doc type'] == 'D1D']) 
            num_foreclosures80 = len(n80[n80['Actual Date'] >= last_3year][n80['doc type'] == 'D1D']) 
            perms_3 = len(permits[permits['ISSUED'] >= last_3year][permits['ISSUED'] <= last_date][permits['sbl'].isin(neighbors_500)])
            try:
                rat = sbl_last_year_500[sbl_last_year_500['Price'] != 0][sbl_last_year_500['princ'] != 0] 
                mean_ratio = rat.apply(lambda x: x['Price'] / x['princ'], axis =1 ).apply(lambda i: i if isinstance(i,float) else np.nan).dropna().mean()

                if not isinstance(mean_ratio,float):
                    mean_ratio = 0
            except Exception as e:
                mean_ratio = 0 
                print(e)

            try:
                my_bill = df[df['SBL'] == sbl]['Principal'].astype(float)
                my_bill = my_bill.iloc[0] if len(my_bill) > 0 else 0  
                std_bill = my_bill - ave_bill_neighbors500
            except:
                std_bill =  0
            try:
                my_bill_last3 = df_last3year[df_last3year['SBL'] == sbl]['Principal'].astype(float) if df_last3year != None else None 
                my_bill_last3 =  my_bill_last3.iloc[0] if my_bill_last3 != None and len(my_bill_last3) > 0 else 0  
                bill_change= (my_bill - my_bill_last3) / my_bill_last3 if my_bill_last3 != -1 and my_bill != -1 else 0 
            except: 
                bill_change = 0

            ave_interest_near_me = df[df['SBL'].isin(neighbors_500)]['Pd Intr'].astype(float).mean()
            try:
                ave_time_deltas =  n500[n500['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas =  ave_time_deltas[ave_time_deltas  >= 0].mean() if len(ave_time_deltas) > 0else 0
                ave_time_deltas300 =  n300[n300['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas300 =  ave_time_deltas300[ave_time_deltas300  >= 0].mean() if len(ave_time_deltas300) > 0 else 0 
                ave_time_deltas80 =  n80[n80['Actual Date'] <= last_date].groupby('sbl').apply(lambda grp: (last_date - grp['Actual Date'].iloc[-1]).days)
                ave_time_deltas80 =  ave_time_deltas80[ave_time_deltas80  >= 0].mean() if len(ave_time_deltas80) > 0 else 0

                ave_p500 = permits[permits['sbl'].isin(neighbors_500)][permits['ISSUED'] <= last_date]['ISSUED'].apply(lambda p: (last_date - p).days).mean()
                ave_p80 = permits[permits['sbl'].isin(neighbors_80)][permits['ISSUED'] <= last_date]['ISSUED'].apply(lambda p: (last_date - p).days).mean()
            except Exception as e:
                ave_time_deltas = 0
                ave_time_deltas300 = 0
                ave_time_deltas80 = 0
                print("OH NO")
                print(e)
                pass 
            try:
                house_length_homeownership = (last_date - data_frame[data_frame['sbl'] == sbl][data_frame['Actual Date'] <= last_date].iloc[-1]['Actual Date']).days
            except:
                house_length_homeownership = 0
            to_dataframe['Average length of homeownership (500m radius)'].append(ave_time_deltas)
            to_dataframe['length of homeownership (this house)'].append(house_length_homeownership)
            to_dataframe['Average bill of neighbors (500m radius) from last year'].append(ave_bill_neighbors500)
            to_dataframe['This house\'s bill last year'].append(my_bill)
            to_dataframe['Average interest of neighbors (500m radius) last year'].append(ave_interest_near_me)
            to_dataframe['Difference between SBL bill and neighbors (500m) last year'].append(std_bill)
            to_dataframe['Average length of homeownership (300m radius)'].append(ave_time_deltas300)
            to_dataframe['This sbl\'s bill percent change from last 3 years ago to last year'].append(bill_change)
            to_dataframe['Average length of homeownership (80m radius)'].append(ave_time_deltas80)
            to_dataframe['Average price sold of neighbors (500m radius) last year'].append(ave_price_sold_last_year_500)
            to_dataframe['Average price sold of neighbors (300m radius) last year'].append(ave_price_sold_last_year_300)
            to_dataframe['Average price sold of neighbors (80m radius) last year'].append(ave_price_sold_last_year_80)
            to_dataframe['Average percent change of house price transaction from last 3 years ago, to last year'].append(ave_price_sold_last_year_500_change)
            to_dataframe['Mean ratio of bill to house price transaction from last year (500m raidus)'].append(mean_ratio)
            to_dataframe['Number of permits last year in a (5oom) radius'].append(len(perm_500_lst_year))
            to_dataframe['Number of permits last 2 years ago in a (500m) radius'].append(len(perm_last_2year500))
            to_dataframe['Number of permits last 3 years ago in a (500m) radius'].append(len(perm_last_3year500))
            to_dataframe['Number of permits last 4-5 years ago in a (500m) radius'].append(len(perm_last_45year500))
            to_dataframe['Number of permits last year in a (80m) radius'].append(len(perm_80_lst_year))
            to_dataframe['Number of permits last 2 years ago in a (80m) radius'].append(len(perm_last_2year80))
            to_dataframe['Number of permits last 3 years ago in a (80m) radius'].append(len(perm_last_3year80))
            to_dataframe['Number of permits last 4-5 years ago in a (80m) radius'].append(len(perm_last_45year80))
            to_dataframe['Mean Value of work (Permits) in a (500m) radius last year'].append(perm_500_lst_year['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (500m) radius last 2 years ago'].append(perm_last_2year500['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (80m) radius last year'].append(perm_80_lst_year['VALUE OF WORK'].mean())
            to_dataframe['Mean Value of work (Permits) in a (80m) radius last 2 years ago'].append(perm_last_2year80['VALUE OF WORK'].mean())
            to_dataframe['Average length of time since the last permits issued in a (500m) radius'].append(ave_p500)
            to_dataframe['Average length of time since the last permits issued in a (80m) radius'].append(ave_p80)
            to_dataframe['Number of foreclosures in a (500m) radius, from last 3 years ago till last year'].append(num_foreclosures500)
            to_dataframe['Number of foreclosures in a (300m) radius, from last 3 years ago till last year'].append(num_foreclosures300)
            to_dataframe['Number of foreclosures in a (80m) radius, from last 3 years ago till last year'].append(num_foreclosures80)
            to_dataframe['Number of Permits from last 3 years ago, to last year, in a (500m) radius'].append(perms_3)
            try:
                neighbor_name  = neigh[sbl]
            except:
                neighbor_name = 'UNKNOWN' 

            # one hot encode neighborhoods 
            for neighborhood in list_of_neighborhoods:
                if neighborhood == neighborhood and neighborhood != neighbor_name:
                    to_dataframe[neighborhood].append(0)
                else: 
                    to_dataframe[neighbor_name].append(1) 
            # this way its just easier to name columns
        last_date = dates
    return pd.DataFrame(to_dataframe)  